In [68]:
import pandas as pd
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing

# Preprocessing and model is used from 
# https://www.kaggle.com/code/jagannathrk/word2vec-cnn-text-classification

In [69]:
import spacy

def tokenization(text):
    """Use this function to tokenize text.

    :param text: Text as list
    :type text: list[spacy.tokens.token.Token]
    :return: Tokenized text as list
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list = []
    for doc in text: 
        # iterate over tokens in docs
        for token in doc:
            token_list.append(token)

    return token_list


def stop_word_removal(text): 
    """Use this function to remove stop words. 

    :param text: Tokens to remove stop words from 
    :type text: list[spacy.tokens.token.Token]
    :return: Tokens without stop words
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list_without_stop = []
    # Don't add token to list if stop word
    for token in text:
        if token.is_stop == False: 
            token_list_without_stop.append(token)

    return token_list_without_stop


def punctutation_removal(text): 
    """Use this function to remove punctuation.

    :param text: Tokens to remove punctuation from
    :type text: list[spacy.tokens.token.Token]
    :return: Tokens without punctuation
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list_no_stop_no_punct = []
    # Don't add token to list if punctuation
    for token in text:
        if token.is_punct == False:
            token_list_no_stop_no_punct.append(token)

    return token_list_no_stop_no_punct


def lemmatization(text): 
    """Use this function to lemmatize a given text.

    :param text: Tokens to lemmatize
    :type text: list[spacy.tokens.token.Token]
    :return: lemmatized tokens
    :rtype: list[str]
    """

    token_list_no_stop_no_punct_lemmatized = []
    for token in text: 
        if "\n" not in token.lemma_:
            token_list_no_stop_no_punct_lemmatized.append(token.lemma_)
    return token_list_no_stop_no_punct_lemmatized


def processing_pipeline(song_data):
    """Use this function to execute the entire processing pipeline on given song data.
    Preprocessing steps:
    - Tokenization
    - Stop word removal
    - Punctuation removal
    - Lemmatization
    - ...

    :param song_data: song data saved in a json file containing song name, artist name and lyrics
    :type song_data: dict
    :return: preprocessed song data
    :rtype: dict
    """

    nlp = spacy.load("en_core_web_sm", disable = ['ner'])
    
    for row in range(len(song_data)):
        text_nlp_pipe = list(nlp.pipe([song_data.iloc[row]["Lyric"]]))
    
        # Tokenization
        song_data.at[row,"Lyric"] = tokenization(text_nlp_pipe)
        # Stop word removal
        song_data.at[row,"Lyric"] = stop_word_removal(song_data.iloc[row]["Lyric"])
        # Punctuation removal
        song_data.at[row,"Lyric"] = punctutation_removal(song_data.iloc[row]["Lyric"])
        # Lemmatization
        song_data.at[row,"Lyric"] = lemmatization(song_data.iloc[row]["Lyric"])
        song_data.at[row,"Lyric"] = " ".join(song_data.iloc[row]["Lyric"])
        song_data.at[row,"Lyric"] = song_data.iloc[row]["Lyric"].lower()
        if row%500 == 0 and row >= 500:
            print(f"processsed: {row} rows out of {len(song_data)}")
    return song_data


In [72]:
df = pd.read_csv('./data/song-data-labels-cleaned.csv')

In [73]:
df.head(2)
df = processing_pipeline(df)
list(df["Lyric"].head(2))

processsed: 500 rows out of 27705
processsed: 1000 rows out of 27705
processsed: 1500 rows out of 27705
processsed: 2000 rows out of 27705
processsed: 2500 rows out of 27705
processsed: 3000 rows out of 27705
processsed: 3500 rows out of 27705
processsed: 4000 rows out of 27705
processsed: 4500 rows out of 27705
processsed: 5000 rows out of 27705
processsed: 5500 rows out of 27705
processsed: 6000 rows out of 27705
processsed: 6500 rows out of 27705
processsed: 7000 rows out of 27705
processsed: 7500 rows out of 27705
processsed: 8000 rows out of 27705
processsed: 8500 rows out of 27705
processsed: 9000 rows out of 27705
processsed: 9500 rows out of 27705
processsed: 10000 rows out of 27705
processsed: 10500 rows out of 27705
processsed: 11000 rows out of 27705
processsed: 11500 rows out of 27705
processsed: 12000 rows out of 27705
processsed: 12500 rows out of 27705
processsed: 13000 rows out of 27705
processsed: 13500 rows out of 27705
processsed: 14000 rows out of 27705
processsed: 

['lie floor feather find hand wanna believe not think s god heaven hell lose lookin sign maybe fool swear spine get shiver like kiss time tho tear stream like river know ask sign scare tho body start shakin feel dry eye tho heart break know love sign sign findin street starin reach turn lookin stranger eye s god heaven hell tt feel like walk cos touch like swear spine get shiver like kiss time tho tear stream like river know ask sign scare tho body start shakin feel dry eye tho heart break know love sign sign lie floor feather find hand wanna believe not think swear spine get shiver like kiss time tho tear stream like river know ask sign scare tho body start shakin feel dry eye tho heart break know love sign sign',
 'oh lord like know think oh oh lord wanna love life heart wonder right try forget near darle miss darling oh bring darle sweet kiss darle think happy think mistake think darling right wrong near darle miss darling oh bring darle sweet kiss darle oh jah oh jah like near oh j

In [74]:
# merge lyrics together
lyrics = []
for i in df['Lyric']:
    lyrics.append(i.split())
print(lyrics[:2])

[['lie', 'floor', 'feather', 'find', 'hand', 'wanna', 'believe', 'not', 'think', 's', 'god', 'heaven', 'hell', 'lose', 'lookin', 'sign', 'maybe', 'fool', 'swear', 'spine', 'get', 'shiver', 'like', 'kiss', 'time', 'tho', 'tear', 'stream', 'like', 'river', 'know', 'ask', 'sign', 'scare', 'tho', 'body', 'start', 'shakin', 'feel', 'dry', 'eye', 'tho', 'heart', 'break', 'know', 'love', 'sign', 'sign', 'findin', 'street', 'starin', 'reach', 'turn', 'lookin', 'stranger', 'eye', 's', 'god', 'heaven', 'hell', 'tt', 'feel', 'like', 'walk', 'cos', 'touch', 'like', 'swear', 'spine', 'get', 'shiver', 'like', 'kiss', 'time', 'tho', 'tear', 'stream', 'like', 'river', 'know', 'ask', 'sign', 'scare', 'tho', 'body', 'start', 'shakin', 'feel', 'dry', 'eye', 'tho', 'heart', 'break', 'know', 'love', 'sign', 'sign', 'lie', 'floor', 'feather', 'find', 'hand', 'wanna', 'believe', 'not', 'think', 'swear', 'spine', 'get', 'shiver', 'like', 'kiss', 'time', 'tho', 'tear', 'stream', 'like', 'river', 'know', 'ask',

In [75]:
# train the word2vec model
# vector size according to https://moj-analytical-services.github.io/NLP-guidance/NNmodels.html#:~:text=The%20standard%20Word2Vec%20pre%2Dtrained,fewer%20dimensions%20to%20represent%20them.
# micount = 2 to prevent misspellings
word2vec_model = Word2Vec(lyrics, vector_size=150, window=5, min_count=2, workers=16)
print(word2vec_model)

Word2Vec<vocab=27165, vector_size=150, alpha=0.025>


In [76]:
# use the keras tokenizer and apply it to the lyrics
# number in first row is vocab size
token = Tokenizer(27165)
token.fit_on_texts(df['Lyric'])
text = token.texts_to_sequences(df['Lyric'])
text = pad_sequences(text, 180)
print(text[3:5])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0    76    19   223   739   251   176   236     9   269  1216   406
    544    34   415     9  3789    52   351    76    16   162    10     1
     34   393    68    36   177    52     2   136    44     9    52    67
   1407   148   778    76    46   422    20    53    16   104   487    52
      2  2007    47     5    55     5    55    76    16   162    10     1
     34   393    68    36   177    52     2   136    44     9    52    67
   1407   148   778    43   206     5   452    16   438     3    16    47
     16    99    76   162    18   162 

In [77]:
# encode the labels
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['Mood'])
y = to_categorical(y)
y[:10]



array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0.]], dtype=float32)

In [78]:
x_train, x_test, y_train, y_test = train_test_split(np.array(text), y, test_size=0.2, stratify=y)

In [79]:
# helper function to use the word2vec as an layer in keras 
# taken from the gensim wikipage: https://github.com/RaRe-Technologies/gensim/wiki/Using-Gensim-Embeddings-with-Keras-and-Tensorflow

from tensorflow.keras.layers import Embedding

def gensim_to_keras_embedding(model, train_embeddings=False):
    """Get a Keras 'Embedding' layer with weights set from Word2Vec model's learned word embeddings.

    Parameters
    ----------
    train_embeddings : bool
        If False, the returned weights are frozen and stopped from being updated.
        If True, the weights can / will be further updated in Keras.

    Returns
    -------
    `keras.layers.Embedding`
        Embedding layer, to be used as input to deeper network layers.

    """
    keyed_vectors = model.wv  # structure holding the result of training
    weights = keyed_vectors.vectors  # vectors themselves, a 2D numpy array    
    index_to_key = keyed_vectors.index_to_key  # which row in `weights` corresponds to which word?

    layer = Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        trainable=train_embeddings,
    )
    return layer

In [80]:
keras_model = Sequential()
keras_model.add(gensim_to_keras_embedding(word2vec_model, True))
keras_model.add(Dropout(0.2))
keras_model.add(Conv1D(50, 5, activation='relu', padding='same', strides=1))
keras_model.add(Conv1D(50, 5, activation='relu', padding='same', strides=1))
keras_model.add(MaxPool1D())
keras_model.add(Dropout(0.2))
keras_model.add(Conv1D(100, 5, activation='relu', padding='same', strides=1))
keras_model.add(Conv1D(100, 5, activation='relu', padding='same', strides=1))
keras_model.add(MaxPool1D())
keras_model.add(Dropout(0.2))
keras_model.add(Conv1D(150, 5, activation='relu', padding='same', strides=1))
keras_model.add(Conv1D(150, 5, activation='relu', padding='same', strides=1))
keras_model.add(GlobalMaxPool1D())
keras_model.add(Dropout(0.2))
keras_model.add(Dense(200))
keras_model.add(Activation('relu'))
keras_model.add(Dropout(0.2))
# Number of moods to be classified to
keras_model.add(Dense(17))
keras_model.add(Activation('softmax'))
keras_model.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')


In [ ]:
keras_model.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_test, y_test))

Epoch 1/3
693/693 [==============================] - 72s 100ms/step - loss: 0.1866 - acc: 0.2750 - val_loss: 0.1916 - val_acc: 0.3043
Epoch 2/3
693/693 [==============================] - 76s 110ms/step - loss: 0.1767 - acc: 0.3139 - val_loss: 0.1757 - val_acc: 0.3337
Epoch 3/3
324/693 [=============>................] - ETA: 37s - loss: 0.1732 - acc: 0.3382

In [ ]:
model.save('./cnn_model_v1')

In [ ]:

import numpy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


# let's create a function that creates the model (required for KerasClassifier) 
# while accepting the hyperparameters we want to tune 
# we also pass some default values such as optimizer='rmsprop'
def create_model(filters,multiplicator, multiplicator2, kernel, adapt_embedding):
    # define model
    
    keras_model = Sequential()
    keras_model.add(gensim_to_keras_embedding(word2vec_model, True))
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(Conv1D(filters, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters*multiplicator, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(Conv1D(filters*multiplicator, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters*multiplicator2, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(Conv1D(filters*multiplicator2, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(GlobalMaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Dense(200))
    keras_model.add(Activation('relu'))
    keras_model.add(Dropout(0.2))
    # Number of moods to be classified to
    keras_model.add(Dense(17))
    keras_model.add(Activation('softmax'))
    
    keras_model.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')
    return keras_model



seed = 7
numpy.random.seed(seed)
batch_size = 32
epochs = 3

model_CV = KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=1)
# define the grid search parameters
filters = [50]
multiplicator = [2]
multiplicator2 = [4]
kernel = [5]
adapt_embedding = [True, False]

param_grid = dict(filters=filters, multiplicator=multiplicator, multiplicator2=multiplicator2, kernel=kernel, adapt_embedding=adapt_embedding)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)


In [ ]:
# print results
print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

In [ ]:

import numpy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


# let's create a function that creates the model (required for KerasClassifier) 
# while accepting the hyperparameters we want to tune 
# we also pass some default values such as optimizer='rmsprop'
def create_model(filters,multiplicator, multiplicator2, kernel, adapt_embedding):
    # define model
    
    keras_model = Sequential()
    keras_model.add(gensim_to_keras_embedding(word2vec_model, True))
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters*multiplicator, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters*multiplicator2, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(GlobalMaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Dense(200))
    keras_model.add(Activation('relu'))
    keras_model.add(Dropout(0.2))
    # Number of moods to be classified to
    keras_model.add(Dense(17))
    keras_model.add(Activation('softmax'))
    
    keras_model.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')
    return keras_model



seed = 7
numpy.random.seed(seed)
batch_size = 32
epochs = 3

model_CV = KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=1)
# define the grid search parameters
filters = [50]
multiplicator = [2]
multiplicator2 = [4]
kernel = [5]
adapt_embedding = [True, False]

param_grid = dict(filters=filters, multiplicator=multiplicator, multiplicator2=multiplicator2, kernel=kernel, adapt_embedding=adapt_embedding)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=2)
grid_result = grid.fit(x_train, y_train)


In [66]:
# print results
print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy for 0.3266558349132538 using {'adapt_embedding': True, 'filters': 50, 'kernel': 5, 'multiplicator': 2, 'multiplicator2': 4}
 mean=0.3267, std=0.001534 using {'adapt_embedding': True, 'filters': 50, 'kernel': 5, 'multiplicator': 2, 'multiplicator2': 4}
 mean=0.3196, std=0.005053 using {'adapt_embedding': False, 'filters': 50, 'kernel': 5, 'multiplicator': 2, 'multiplicator2': 4}
